In [1]:
#Donald Dunagan and Andrew Stout
#811-648-053
#10 November 2018
#CSCI 6560 Term Project
#The use of evolutionary computation in the
#optimization of neural networks

# Utilizing the DEAP Package: Distributed Evolutionary Algorithms in Python

In [2]:
#    DEAP is free software: you can redistribute it and/or modify
#    it under the terms of the GNU Lesser General Public License as
#    published by the Free Software Foundation, either version 3 of
#    the License, or (at your option) any later version.
#
#    DEAP is distributed in the hope that it will be useful,
#    but WITHOUT ANY WARRANTY; without even the implied warranty of
#    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
#    GNU Lesser General Public License for more details.
#
#    You should have received a copy of the GNU Lesser General Public
#    License along with DEAP. If not, see <http://www.gnu.org/licenses/>.

# Set Up Environment

In [13]:
#imports

import numpy as np
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.utils import shuffle
from sklearn.metrics import make_scorer, mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, GridSearchCV
from scipy import stats
import random, operator, time, itertools, math, numpy
from deap import algorithms, base, creator, tools
import warnings
import array

In [4]:
#import Parkinson's Dataset and visualize

PDDF =  pd.read_csv("C:\\Users\\gray\\Desktop\\Evolutionary_Computing\\Term_Project\\Parkinson's_Data.csv")
print ('The shape of the dataframe is:',PDDF.shape)
PDDF.head()

The shape of the dataframe is: (5875, 22)


,subject#,age,sex,test_time,motor_UPDRS,total_UPDRS,Jitter(%),Jitter(Abs),Jitter:RAP,Jitter:PPQ5,...,Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,Shimmer:DDA,NHR,HNR,RPDE,DFA,PPE
0,1,72,0,5.6431,28.199,34.398,0.00662,0.000034,0.00401,0.00317,...,0.230,0.01438,0.01309,0.01662,0.04314,0.014290,21.640,0.41888,0.54842,0.16006
1,1,72,0,12.6660,28.447,34.894,0.00300,0.000017,0.00132,0.00150,...,0.179,0.00994,0.01072,0.01689,0.02982,0.011112,27.183,0.43493,0.56477,0.10810
2,1,72,0,19.6810,28.695,35.389,0.00481,0.000025,0.00205,0.00208,...,0.181,0.00734,0.00844,0.01458,0.02202,0.020220,23.047,0.46222,0.54405,0.21014
3,1,72,0,25.6470,28.905,35.810,0.00528,0.000027,0.00191,0.00264,...,0.327,0.01106,0.01265,0.01963,0.03317,0.027837,24.445,0.48730,0.57794,0.33277
4,1,72,0,33.6420,29.187,36.375,0.00335,0.000020,0.00093,0.00130,...,0.176,0.00679,0.00929,0.01819,0.02036,0.011625,26.126,0.47188,0.56122,0.19361


#### Initial shuffling and splitting

In [5]:
#shuffle the data
PDDF = shuffle(PDDF)

#isolate features
features_to_drop = ['subject#','age','sex','test_time','motor_UPDRS','total_UPDRS']
just_features = PDDF.drop(features_to_drop,axis=1)
    
#isolate motor_UPDRS
just_motor = PDDF.motor_UPDRS
   
#isolate total_UPDRS
just_total = PDDF.total_UPDRS

#split into training and testing data
training_features = just_features[:4700]
testing_features = just_features[4700:]
    
training_motor = just_motor[:4700]
testing_motor = just_motor[4700:]
    
training_total = just_total[:4700]
testing_total = just_total[4700:] 

#### A function for shuffling and splitting the data

In [6]:
def shuffleSplit():
    '''
    Shuffles and splits the DataFrame 80%-20%
    '''
    global PDDF
    
    #shuffle the data
    PDDF = shuffle(PDDF)

    #isolate features
    features_to_drop = ['subject#','age','sex','test_time','motor_UPDRS','total_UPDRS']
    global just_features
    just_features = PDDF.drop(features_to_drop,axis=1)
    
    #isolate motor_UPDRS
    global just_motor
    just_motor = PDDF.motor_UPDRS
   
    #isolate total_UPDRS
    global just_total
    just_total = PDDF.total_UPDRS

    #split into training and testing data
    global training_features
    training_features = just_features[:4700]
    global testing_features
    testing_features = just_features[4700:]
    
    global training_motor
    training_motor = just_motor[:4700]
    global testing_motor
    testing_motor = just_motor[4700:]
    
    global training_total
    training_total = just_total[:4700]
    global testing_total
    testing_total = just_total[4700:]  
    
    return

In [7]:
#cross validation needs to return the mean absolute error (MAE) for each fold
#so i have to make a custom scoring object

#create scorer object
myScorer = make_scorer(mean_absolute_error)

In [8]:
PDDF.head()

,subject#,age,sex,test_time,motor_UPDRS,total_UPDRS,Jitter(%),Jitter(Abs),Jitter:RAP,Jitter:PPQ5,...,Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,Shimmer:DDA,NHR,HNR,RPDE,DFA,PPE
1306,9,68,0,56.364,17.0000,24.225,0.00733,0.000059,0.00420,0.00356,...,0.239,0.00969,0.01179,0.01705,0.02906,0.019408,23.531,0.51433,0.73652,0.286340
926,7,72,0,133.320,15.5710,22.085,0.00399,0.000038,0.00154,0.00228,...,0.188,0.00962,0.01331,0.02072,0.02885,0.009184,22.737,0.46979,0.59728,0.186580
3034,22,57,1,124.880,9.6352,10.765,0.00198,0.000005,0.00100,0.00115,...,0.125,0.00683,0.00721,0.00858,0.02050,0.001880,31.746,0.27155,0.53508,0.096121
1994,15,65,0,54.469,11.7890,16.387,0.00404,0.000029,0.00175,0.00169,...,0.268,0.01772,0.01804,0.02631,0.05315,0.012617,25.899,0.60989,0.64045,0.142060
3640,27,57,1,98.467,11.5590,15.559,0.00516,0.000030,0.00268,0.00260,...,0.242,0.01500,0.01452,0.02086,0.04499,0.007013,24.952,0.56507,0.74094,0.213010


# Scikit Learn MLP

## Baselines for untuned, default MLP regressors

### 5-fold cross validated training MAE of an untuned MLP regressor targeting motor_UPDRS

In [9]:
#5-fold cross validated untuned MLP targeting motor_UPDRS

MLP = MLPRegressor()

#5-fold CV
vals = cross_val_score(MLP,training_features,training_motor,cv=5, scoring=myScorer)
print("MAE: {}".format(vals.mean()))

MAE: 6.956135109977886


### 5-fold cross validated training MAE of an untuned MLP regressor targeting total_UPDRS

In [40]:
#5-fold cross validated untuned MLP targeting total_UPDRS

MLP = MLPRegressor()

#5-fold CV
vals = cross_val_score(MLP,training_features,training_total,cv=5, scoring=myScorer)
print("MAE: {}".format(vals.mean()))

MAE: 8.594094070273957


### Idea is to use evolutionary computation to optimize MLP hyperparameters
    *Fix weight solver to 'adam' 
    
    
    
    - Hyperparameters to tune: 
        - Neurons in the first hidden layer
        - Neurons in the second hidden layer
        - Activation function for the hidden layers
        - Algorithm for weight optimization
        - Alpha: L2 penalty parameter
    - Fitness evaluation:
        - round floats to ints (if needed)
        - convert ints to the values they represent (if needed)
        - Train NN on training data (80%)
        - Make predictions using testing data (20%)
        - Fitness is mean abosolute error (MAE) between predicted and actual values
    - Genotype:
        - float string of length 6
    - Phenotype (hyperparameters for NN):
        - size_layer1
            - number of neurons in the first hidden layer
        - size_layer2
            - number of neurons in the second hidden layer
        - activaton_function
            - 0 : identity, 1: logistic, 2: tanh, 3: relu
        - weight fittting algorithm
            - 0: lbfgs, 1: sgd, 2: adam
        - alpha
            - real value between 0.0001 and 1000

### Fitness Evaluation Functon for Motor UPDRS

In [42]:
'''
size_layer1, activation_function, optimization_algorithm, alpha value, size_layer2
'''
def MLP_Eval_Motor(vals):
    
    #genotypes is real valued so some alleles need to be 
    #rounded to nearest int
    
    vals[0] = round(vals[0])
    vals[1] = round(vals[1])
    vals[2] = round(vals[2])
    vals[4] = round(vals[4])
    
    #if params are out of bounds, place them back in bounds
    if vals[0] <= 0:
        vals[0] = 1        #size of hidden layer1
    if vals[0] > 100:
        vals[0] = 100
        
    if vals[4] <= 0:
        vals[4] = 1        #size of hidden layer2
    if vals[4] > 100:
        vals[4] = 100 

    if vals[1] < 0:
        vals[1] = 0     #activation function
    if vals[1] > 3:
        vals[1] = 3
        
    if vals[2] < 0:
        vals[2] = 0   #optimization algorithm
    if vals[2] > 2:
        vals[2] = 2
        
    if vals[3] < 0.0001:
        vals[3] = .0001
    if vals[3] > 1000:
        vals[3] = 1000
        
    #assign vals to int alleles (if necessary)
    if vals[1] == 0:
        AF = 'identity'
    if vals[1] == 1:
        AF = 'logistic'    #activation function
    if vals[1] == 2:
        AF = 'tanh'
    if vals[1] == 3:
        AF = 'relu'

    if vals[2] == 0:
        OA = 'lbfgs'
    if vals[2] == 1:
        OA = 'sgd'    #optimization algorithm
    if vals[2] == 2:
        OA = 'adam'
    
    #construct phenotype
    MLP = MLPRegressor(hidden_layer_sizes=(vals[0],vals[4],), activation=AF, solver=OA, alpha = vals[3])

    #some phenotypes will throw an error
    try:
        #train NN
        MLP.fit(training_features, training_motor)
        #make predictions using testing data
        predicts = (MLP.predict(testing_features)).astype(float)
        #compare predicts and actual and record mean absolute error
        MAE = mean_absolute_error(testing_motor,predicts)
        goodPredicts = True
    #if there is an error in fitting or evaluating the MLP,
    #this candidate solution is clearly bad
    except:
        MAE = 1000
    
    #return MAE as fitness
    return (MAE,)

### Fitness Evaluation Functon for Total UPDRS

In [43]:
'''
size_layer1, activation_function, optimization_algorithm, alpha value, size_layer2
'''
def MLP_Eval_Total(vals):
    
    #genotypes is real valued so some alleles need to be 
    #rounded to nearest int
    
    vals[0] = round(vals[0])
    vals[1] = round(vals[1])
    vals[2] = round(vals[2])
    vals[4] = round(vals[4])
    
    #if params are out of bounds, place them back in bounds
    if vals[0] <= 0:
        vals[0] = 1        #size of hidden layer1
    if vals[0] > 100:
        vals[0] = 100
        
    if vals[4] <= 0:
        vals[4] = 1        #size of hidden layer2
    if vals[4] > 100:
        vals[4] = 100 

    if vals[1] < 0:
        vals[1] = 0     #activation function
    if vals[1] > 3:
        vals[1] = 3
        
    if vals[2] < 0:
        vals[2] = 0   #optimization algorithm
    if vals[2] > 2:
        vals[2] = 2
        
    if vals[3] < 0.0001:
        vals[3] = .0001
    if vals[3] > 1000:
        vals[3] = 1000
        
    #assign vals to int alleles (if necessary)
    if vals[1] == 0:
        AF = 'identity'
    if vals[1] == 1:
        AF = 'logistic'    #activation function
    if vals[1] == 2:
        AF = 'tanh'
    if vals[1] == 3:
        AF = 'relu'

    if vals[2] == 0:
        OA = 'lbfgs'
    if vals[2] == 1:
        OA = 'sgd'    #optimization algorithm
    if vals[2] == 2:
        OA = 'adam'
    
    #construct phenotype
    MLP = MLPRegressor(hidden_layer_sizes=(vals[0],vals[4],), activation=AF, solver=OA, alpha = vals[3])

    #some phenotypes will throw an error
    try:
        #train NN
        MLP.fit(training_features, training_total)
        #make predictions using testing data
        predicts = (MLP.predict(testing_features)).astype(float)
        #compare predicts and actual and record mean absolute error
        MAE = mean_absolute_error(testing_total,predicts)
        goodPredicts = True
    #if there is an error in fitting or evaluating the MLP,
    #this candidate solution is clearly bad
    except:
        MAE = 1000
    
    #return MAE as fitness
    return (MAE,)

### Function for Generating Individuals

In [21]:
#generates an individual with random number of nodes in hidden layer 1,
#random number of nodes in hidden layer 2,
#random activation function, and random weight optimizer 
#within the restrictions
def Generate_Individual():
    individual = [random.uniform(1,101),random.uniform(0,4), random.uniform(0,3), random.uniform(0.001,1000),random.uniform(0,101)]
    return individual

In [23]:
indiv = Generate_Individual()
print(indiv)
MLP_Eval(indiv)

[95.12069331665627, 0.7205813124836169, 1.70550157651737, 821.916873965368, 40.901309260890486]


C:\Users\gray\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


(8.694249991467276,)

### Create FitnessMin and Individual classes

In [45]:
#python throws a warning due to an intentional duplicate class
import warnings
warnings.filterwarnings('ignore')

In [24]:
#the fitness function is to be minimized
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
#an individual is composed of a list of vals with fitness to be minimized
creator.create("Individual", list, fitness=creator.FitnessMin)

### Toolbox

In [46]:
#instantiate toolbox
toolbox = base.Toolbox()
#tool to create an individual
toolbox.register("individual", tools.initIterate, creator.Individual, Generate_Individual)
#tool to create a population
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
#tool for fitness evaluation
toolbox.register("evaluate", MLP_Eval_Motor)

#tool for crossover (one point)
toolbox.register("mate", tools.cxOnePoint)
#mutation is gaussian mutation
toolbox.register("mutate", tools.mutGaussian, mu=0,sigma=1,indpb=0.1)
#tool for tournament selection
toolbox.register("select", tools.selTournament, tournsize=3)

In [47]:
#run the algorithm

#generate the intial population (10 * number of dimensions)
pop = toolbox.population(n=20)
#Object for recording best candidate to ever live
HOF = tools.HallOfFame(1)
#run the algorithm
result, log = algorithms.eaSimple(pop, toolbox,halloffame=HOF, cxpb=0.7, mutpb=1, ngen=10, verbose=True)

C:\Users\gray\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


gen	nevals
0  	20    
1  	20    
2  	20    
3  	20    
4  	20    
5  	20    
6  	20    
7  	20    
8  	20    
9  	20    
10 	20    


In [48]:
result

[[24, 2, 0, 54.576964869021865, 5],
 [24, 2, 0, 54.576964869021865, 54],
 [25, 2, 0, 52.67023688866282, 4],
 [24, 2, 0, 148.14067778104987, 6],
 [24, 3, 0, 54.576964869021865, 5],
 [24, 2, 0, 54.576964869021865, 5],
 [24, 2, 0, 52.67023688866282, 5],
 [24, 2, 0, 54.576964869021865, 54],
 [24, 2, 0, 148.14067778104987, 6],
 [24, 2, 0, 148.500108572079, 6],
 [24, 2, 0, 147.2682134442826, 72],
 [24, 2, 0, 54.576964869021865, 54],
 [24, 2, 0, 52.67023688866282, 5],
 [24, 1, 0, 54.576964869021865, 5],
 [24, 2, 0, 54.576964869021865, 5],
 [24, 2, 0, 148.500108572079, 6],
 [24, 2, 0, 148.500108572079, 6],
 [24, 2, 0, 54.576964869021865, 54],
 [25, 2, 0, 54.576964869021865, 5],
 [24, 2, 0, 54.576964869021865, 54]]

In [9]:

MLP_Eval(HOF.items[0])

NameError: name 'MLP_Eval' is not defined

# Evolving NN Weights

## 5-Fold Cross Validation to Determine Optimum Number of Hidden Nodes 

#### Compare 8 nodes in hidden layer to 128

In [56]:
#5-fold cross validated untuned MLP targeting motor_UPDRS

MLP = MLPRegressor(hidden_layer_sizes=(8,))

#5-fold CV
vals = cross_val_score(MLP,training_features,training_motor,cv=5, scoring=myScorer)
print("The 5-fold CV'd MAE with 8 nodes in the hidden layer is: {}".format(vals.mean()))

C:\Users\gray\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\gray\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\gray\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


The 5-fold CV'd MAE with 8 nodes in the hidden layer is: 6.974299956063492


#### ^ the NN is not converging with only 8 hidden nodes. Needs more

In [59]:
#5-fold cross validated untuned MLP targeting motor_UPDRS

MLP = MLPRegressor(hidden_layer_sizes=(128,))

#5-fold CV
vals = cross_val_score(MLP,training_features,training_motor,cv=5, scoring=myScorer)
print("The 5-fold CV'd MAE with 128 nodes in the hidden layer is: {}".format(vals.mean()))

The 5-fold CV'd MAE with 128 nodes in the hidden layer is: 6.907500972834988


#### 128 converges and has better MAE, but can we get the same result with less nodes?

In [62]:
#5-fold cross validated untuned MLP targeting motor_UPDRS

MLP = MLPRegressor(hidden_layer_sizes=(64,))

#5-fold CV
vals = cross_val_score(MLP,training_features,training_motor,cv=5, scoring=myScorer)
print("The 5-fold CV'd MAE with 64 nodes in the hidden layer is: {}".format(vals.mean()))

The 5-fold CV'd MAE with 64 nodes in the hidden layer is: 6.907716044950663


#### Accuracy is the same with half the nodes. Let's try 32

In [67]:
#5-fold cross validated untuned MLP targeting motor_UPDRS

MLP = MLPRegressor(hidden_layer_sizes=(32,))

#5-fold CV
vals = cross_val_score(MLP,training_features,training_motor,cv=5, scoring=myScorer)
print("The 5-fold CV'd MAE with 32 nodes in the hidden layer is: {}".format(vals.mean()))

The 5-fold CV'd MAE with 32 nodes in the hidden layer is: 6.934125781786435


#### Accuracy is worse. 64 nodes in the hidden layer is our sweet spot

## Evolving NN Weights with a Genetic Algorithm

#### Evaluation Function

In [17]:
def evalNN(individual):
    """
    Takes the actual UPDRS values (either motor or total) and an np 
    array of length 1088, extracts the weight matrices,
    runs the neural network, and calculates the mean absolute 
    error between predicted UPDRS and actual UPDRS
    """
    #extract weight matrices
    weights1 = np.array(individual[0:1024]).reshape(16,64)
    weights2 = np.array(individual[1024:]).reshape(64,1)
    
    
    #perform matrix operations that take place inside the 
    #neural network
    
    #matrix multiplication between feature matrix and weights of hidden layer
    hidden = np.matmul(training_features,weights1)
    #add biases of hidden layer to this
    hidden = hidden + 1
    
    #perform ReLu activation function
    np.maximum(hidden,0,hidden)
    
    #matrix multiplication between the result of the hidden layer and 
    #the weights of the output layer
    final = np.matmul(hidden,weights2)

    #activation function for output node of regression is linear
    #so this is result of the neural network
    
    #calculate mean absolute error between predicted and actual and return 
    MAE =  mean_absolute_error(training_motor,final)
    return (MAE,)

#### Function for generating an individual

In [98]:
def Generate_Individual():
    #individuals generated from the standard normal distribution: mu=0, std=1
    individual = np.random.normal(0,1,1088)
    return individual

#### Create FitnessMin and Individual classes

In [99]:
#the fitness function is to be minimized
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
#an individual is composed of a list of vals with fitness to be minimized
creator.create("Individual", list, fitness=creator.FitnessMin)

C:\Users\gray\Anaconda3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\gray\Anaconda3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


#### Toolbox

In [100]:
#instantiate toolbox
toolbox = base.Toolbox()
#tool to create an individual
toolbox.register("individual", tools.initIterate, creator.Individual, Generate_Individual)
#tool to create a population
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
#tool for fitness evaluation
toolbox.register("evaluate", evalNN)

#tool for crossover (one point)
toolbox.register("mate", tools.cxOnePoint)
#mutation is gaussian mutation
toolbox.register("mutate", tools.mutGaussian, mu=0,sigma=1.0,indpb=(1/1088))
#tool for tournament selection
toolbox.register("select", tools.selTournament, tournsize=50)

In [101]:
evalNN(toolbox.individual())

(159.63182079007703,)

In [104]:
ini = toolbox.individual()
print (max(ini))
print (min(ini))

2.8745819325491113
-2.8234171785759448


#### Run the algorithm

In [92]:
#run the algorithm

#generate the intial population (10 * number of dimensions)
pop = toolbox.population(n=3000)
#Object for recording best candidate to ever live
HOF = tools.HallOfFame(1)
#run the algorithm
result, log = algorithms.eaSimple(pop, toolbox,halloffame=HOF, cxpb=0.7, mutpb=1, ngen=5, verbose=True)

gen	nevals
0  	3000  
1  	3000  
2  	3000  
3  	3000  
4  	3000  
5  	3000  


In [93]:
print (max(HOF.items[0]))
print (min(HOF.items[0]))
evalNN(HOF.items[0])

4.041972827555817
-2.5098375082051225


(6.879379425004331,)

## Evolve NN Weights with Evolution Strategies

### Creator

In [94]:
#fitness is to be minimized, an individual is an array of doubles, a strategy is an array of doubles
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", array.array, typecode="d", fitness=creator.FitnessMin, strategy=None)
creator.create("Strategy", array.array, typecode="d")

C:\Users\gray\Anaconda3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\gray\Anaconda3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\gray\Anaconda3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Strategy' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


### Function for Generating Individuals

In [139]:
#generates object variables and strategy parameters for an individual
def generateES(icls, scls):
    ind = icls(random.uniform(-3, 3) for _ in range(1088))
    ind.strategy = scls(random.uniform(.001, 3) for _ in range(1088))
    return ind

### Function for Ensuring Sigmas are Valid

In [140]:
def checkStrategy(minstrategy):
    def decorator(func):
        def wrappper(*args, **kargs):
            children = func(*args, **kargs)
            for child in children:
                for i, s in enumerate(child.strategy):
                    if s < minstrategy:
                        child.strategy[i] = minstrategy
            return children
        return wrappper
    return decorator

### Toolbox

In [141]:
toolbox = base.Toolbox()
toolbox.register("individual", generateES, creator.Individual, creator.Strategy)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
#two point crossover on both individuals and their strategies
toolbox.register("mate", tools.cxESTwoPoint)
toolbox.register("mutate", tools.mutESLogNormal, c=1.0, indpb=0.03)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evalNN)

toolbox.decorate("mate", checkStrategy(.01))
toolbox.decorate("mutate", checkStrategy(.01))

### Run the Algorithm

In [142]:
pop = toolbox.population(100)
HOF = tools.HallOfFame(1)

pop, logbook = algorithms.eaMuCommaLambda(pop, toolbox, mu=100, lambda_=250, 
    cxpb=0.6, mutpb=0.3, ngen=20, halloffame=HOF)

gen	nevals
0  	100   
1  	228   
2  	217   
3  	219   
4  	231   
5  	226   
6  	221   
7  	221   
8  	220   
9  	225   
10 	226   
11 	222   
12 	218   
13 	229   
14 	223   
15 	223   
16 	221   
17 	226   
18 	225   
19 	226   
20 	223   


In [143]:
print (max(HOF.items[0]))
print (min(HOF.items[0]))
evalNN(HOF.items[0])

5.741582144462124
-7.71054071500519


(6.931020919272436,)